In [1]:
import pandas as pd
import smtplib
from pathlib import Path
import os
from email.message import EmailMessage

In [30]:
def EnviarEmail(destino, assunto, mensagem):
    server = smtplib.SMTP('smtp.gmail.com: 587')
    server.starttls()
    server.login('NotaFiscal@unicacontabil.com', 'nfunica123')

    email= EmailMessage()
    email['From'] = 'NotaFiscal@unicacontabil.com'
    email['To'] = destino
    #email['To'] = 'evelyn@unicacontabil.com'
    email['Cc'] = 'fiscal@unicacontabil.com; evelyn@unicacontabil.com'
    email['Subject'] = assunto

    email.set_content(mensagem)

    server.send_message(email)
    print('E-mail para {} enviado com sucesso!' .format(destino))

In [31]:
path_download = os.path.join(Path.home(), "Downloads")

# arq = 'C:/Users/Henrique/Downloads/ALÍQUOTAS MENSAIS.xlsx'
arq = os.path.join(path_download, 'ALÍQUOTAS MENSAIS.xlsx')


df = pd.read_excel(arq, header=2)
df2 = df.query("OBSERVAÇÕES != 'NÃO ENVIAR POR E-MAIL'").iloc[:, 0:29]
df2.set_index('CNPJ', inplace=True)

In [32]:
#Empresas Unicas
campos = "EMPRESA;EMAIL 01".split(';')
df3 = df2[campos].drop_duplicates(keep="first")

In [33]:
#Dicionarios Auxiliares
dMeses = {
    1:  ['Janeiro',     [1,3,4]],
    2:  ['Fevereiro',   [1,5,6]],
    3:  ['Março',       [1,7,8]],
    4:  ['Abril',       [1,9,10]],
    5:  ['Maio',        [1,11,12]],
    6:  ['Junho',       [1,13,14]],
    7:  ['Julho',       [1,15,16]],
    8:  ['Agosto',      [1,17,18]],
    9:  ['Setembro',    [1,19,20]],
    10: ['Outubro',     [1,21,22]],
    11: ['Novembro',    [1,23,24]],
    12: ['Dezembro',    [1,25,26]],
}

d = {
    'ANEXO 1' : ['Comércio', 'Alíquota ICMS: '],
    'ANEXO 2' : ['Industria', 'Alíquota ICMS: '],
    'ANEXO 3' : ['Serviço (III)', 'Alíquota ISS: '],
    'ANEXO 4' : ['Serviço (IV)', 'Alíquota ISS: '],
    'ANEXO 5' : ['Serviço (V)', 'Alíquota ISS: '],
    'FATOR R' : ['Serviço', 'Alíquota ISS: ']
}

In [34]:
# Main
for x,y in dMeses.items():
    print('ID: {} - Mes: {}' .format(x, y[0]))

mes = None
while mes is None:
    try:
        iMes = input('Digite o ID do mes selecionado: ')
        mes = dMeses[int(iMes)][0]
        cols = dMeses[int(iMes)][1]
    except:
        pass
print('{} / {}' .format(mes, cols) )
#Seleciona qual mes sera enviado o Email
df4 = df2.iloc[:,cols]

ID: 1 - Mes: Janeiro
ID: 2 - Mes: Fevereiro
ID: 3 - Mes: Março
ID: 4 - Mes: Abril
ID: 5 - Mes: Maio
ID: 6 - Mes: Junho
ID: 7 - Mes: Julho
ID: 8 - Mes: Agosto
ID: 9 - Mes: Setembro
ID: 10 - Mes: Outubro
ID: 11 - Mes: Novembro
ID: 12 - Mes: Dezembro
Julho / [1, 15, 16]


In [35]:
selecao = df3['EMAIL 01'].isnull()
# df3[selecao]

df3 = df3[~selecao]
df3["EMAIL 01"].replace(',' , ';' , regex=True, inplace=True)

In [28]:
# df4.info()
df3

,EMPRESA,EMAIL 01
CNPJ,,
2.816376e+12,ACF MARISTA LTDA,acfmarista@uol.com.br; rogeriorosacosta@gmail.com
1.485999e+13,BIZ CENTER LTDA,euripedes@bizcenter.com.br; comercial@bizcente...
2.666956e+13,BIOMEDIC LABORATORIO DE ANALISES CLINICAS LTDA,admlabnsaparecida@outlook.com; labbiomedic@hot...


In [36]:
ano = 2022
msgHead = """
Bom Dia,

Segue abaixo a alíquota do mês de {}/{}.
""" .format(mes, ano)

msgFooter = """


Atenciosamente;

Única Contábil
Departamento Fiscal
(62) 3212-1200 | (62) 98228-0118 
"""   

# path = "C:/Temp/Unica/"
#Loop
for index, row1 in df3.iterrows(): 
    dfTemp = df4[(df4.index == index)]

    Title = 'ALÍQUOTA MENSAL ({}/{}) - ' .format(mes, ano) + row1["EMPRESA"] 
    EmailList = row1['EMAIL 01'] 

    msgBody = ""
    for index, row in dfTemp.iterrows(): 
        msgBody = msgBody + "\n"
        msgBody = msgBody + d[row['ANEXO SIMPLES']][0] + "\n"
        msgBody = msgBody + d[row['ANEXO SIMPLES']][1] + '{0:.2f}%'.format(row.iloc[1]*100) + "\n"
        msgBody = msgBody + 'Alíquota efetiva total: ' '{0:.2f}%'.format(row.iloc[2]*100) + "\n" 

    msgBody = msgBody.replace(".", ",")

    #Inicia o procedimento de envio de email
    msg = msgHead + msgBody + msgFooter
    EnviarEmail(EmailList, Title, msg)

    # Gravar .txt 
    # f = open(path + row1["EMPRESA"] +".txt", "x")
    # f.write(Title)
    # f.write(EmailList)

    # f.write(msg)
    # f.close()



E-mail para acfmarista@uol.com.br; rogeriorosacosta@gmail.com enviado com sucesso!
E-mail para euripedes@bizcenter.com.br; comercial@bizcenter.com.br enviado com sucesso!
E-mail para admlabnsaparecida@outlook.com; labbiomedic@hotmail.com enviado com sucesso!
E-mail para agencia84@gmail.com enviado com sucesso!
E-mail para radiogyn@uol.com.br enviado com sucesso!
E-mail para g_scartezini@yahoo.com.br enviado com sucesso!
E-mail para alexandro@amgcorp.com.br; alexandroolima@gmail.com enviado com sucesso!
E-mail para financeiro@termosularcondicionado.com.br; termosul@gmail.com enviado com sucesso!
E-mail para financeiro@multiodontogo.com.br; mkt_rh@multiodontogo.com.br enviado com sucesso!
E-mail para drdanielitikawa@gmail.com enviado com sucesso!
E-mail para simone@admiralservicos.com.br enviado com sucesso!
E-mail para simone@admiralservicos.com.br enviado com sucesso!
E-mail para financeirobrgaap@gmail.com enviado com sucesso!
E-mail para odontoespatheneu@gmail.com; fhenriquef123@gmail